# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [15]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
from panel.interact import interact
from panel import widgets
import os
from pathlib import Path
from dotenv import load_dotenv

In [2]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [3]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("MAPBOX")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [4]:
# Import the CSVs to Pandas DataFrames
file_path = Path("toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path)

In [5]:
to_data.head()

,neighbourhood,single_detached_house,apartment_five_storeys_plus,movable_dwelling,semi_detached_house,row_house,duplex,apartment_five_storeys_less,other_house,average_house_value,shelter_costs_owned,shelter_costs_rented
year,,,,,,,,,,,,
2001,Agincourt North,3715,1480,0,1055,1295,195,185,5,200388,810,870
2001,Agincourt South-Malvern West,3250,1835,0,545,455,105,425,0,203047,806,892
2001,Alderwood,3175,315,0,470,50,185,370,0,259998,817,924
2001,Annex,1060,6090,5,1980,605,275,3710,165,453850,1027,1378
2001,Banbury-Don Mills,3615,4465,0,240,380,15,1360,0,371864,1007,1163


In [6]:
df_neighbourhood_locations.head()

,neighbourhood,lat,lon
0,Agincourt North,43.805441,-79.266712
1,Agincourt South-Malvern West,43.788658,-79.265612
2,Alderwood,43.604937,-79.541611
3,Annex,43.671585,-79.404001
4,Banbury-Don Mills,43.737657,-79.349718


- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [10]:
# Getting the data from the top 10 expensive neighbourhoods
top_10_neighborhoods = to_data.groupby(['neighbourhood']).max().sort_values(by=['average_house_value'], ascending=False).head(10).reset_index()
# Calculate the mean number of dwelling types units per year
mean_dwelling_units_by_year = to_data.groupby(['year']).mean()
# Calculate the average monthly shelter costs for owned and rented dwellings
average_shelter_cost = to_data.groupby(['year']).mean()[['shelter_costs_owned', 'shelter_costs_rented']]

In [8]:
mean_dwelling_units_by_year

,single_detached_house,apartment_five_storeys_plus,movable_dwelling,semi_detached_house,row_house,duplex,apartment_five_storeys_less,other_house,average_house_value,shelter_costs_owned,shelter_costs_rented
year,,,,,,,,,,,
2001,2149.500000,2535.821429,0.535714,649.964286,373.964286,169.892857,835.000000,21.714286,289882.885714,846.878571,1085.935714
2006,1906.142857,2710.000000,1.178571,495.928571,390.642857,314.964286,1163.214286,9.535714,424059.664286,1316.800000,925.414286
2011,1963.857143,3065.857143,0.714286,517.714286,431.107143,319.642857,1170.678571,15.464286,530424.721429,1448.214286,1019.792857
2016,1926.285714,3523.357143,0.678571,508.571429,439.750000,347.035714,1182.678571,20.321429,664068.328571,1761.314286,1256.321429


In [24]:
yearly_dwelling_units = pd.read_csv("Grouped_yearly_dwelling_stats.csv", index_col='year')
yearly_dwelling_count_averaged_cost_removed = yearly_dwelling_units.drop(['average_house_value', 'shelter_costs_owned', 'shelter_costs_rented'], axis=1)
yearly_dwelling_count_averaged_cost_removed

,single_detached_house,apartment_five_storeys_plus,movable_dwelling,semi_detached_house,row_house,duplex,apartment_five_storeys_less,other_house
year,,,,,,,,
2001,300930,355015,75,90995,52355,23785,116900,3040
2006,266860,379400,165,69430,54690,44095,162850,1335
2011,274940,429220,100,72480,60355,44750,163895,2165
2016,269680,493270,95,71200,61565,48585,165575,2845


In [12]:
to_average_data = to_data.groupby(['neighbourhood']).mean().reset_index()
to_average_data.head()

,neighbourhood,single_detached_house,apartment_five_storeys_plus,movable_dwelling,semi_detached_house,row_house,duplex,apartment_five_storeys_less,other_house,average_house_value,shelter_costs_owned,shelter_costs_rented
0,Agincourt North,3435.00,1947.50,2.50,863.75,1406.25,512.50,547.50,10.00,329811.5,1109.00,983.50
1,Agincourt South-Malvern West,2897.50,2180.00,1.25,375.00,456.25,523.75,628.75,32.50,334189.0,1131.25,985.00
2,Alderwood,2903.75,302.50,1.25,503.75,76.25,302.50,502.50,1.25,427922.5,1166.75,1003.25
3,Annex,751.25,7235.00,1.25,1375.00,613.75,355.00,4605.00,83.75,746977.0,1692.75,1315.25
4,Banbury-Don Mills,3572.50,5388.75,1.25,273.75,626.25,32.50,1340.00,0.00,612039.0,1463.50,1242.75


In [13]:
to_average_data_merged_location = pd.merge(left=to_average_data, right=df_neighbourhood_locations, on='neighbourhood', how='inner')
to_average_data_merged_location.head()

,neighbourhood,single_detached_house,apartment_five_storeys_plus,movable_dwelling,semi_detached_house,row_house,duplex,apartment_five_storeys_less,other_house,average_house_value,shelter_costs_owned,shelter_costs_rented,lat,lon
0,Agincourt North,3435.00,1947.50,2.50,863.75,1406.25,512.50,547.50,10.00,329811.5,1109.00,983.50,43.805441,-79.266712
1,Agincourt South-Malvern West,2897.50,2180.00,1.25,375.00,456.25,523.75,628.75,32.50,334189.0,1131.25,985.00,43.788658,-79.265612
2,Alderwood,2903.75,302.50,1.25,503.75,76.25,302.50,502.50,1.25,427922.5,1166.75,1003.25,43.604937,-79.541611
3,Annex,751.25,7235.00,1.25,1375.00,613.75,355.00,4605.00,83.75,746977.0,1692.75,1315.25,43.671585,-79.404001
4,Banbury-Don Mills,3572.50,5388.75,1.25,273.75,626.25,32.50,1340.00,0.00,612039.0,1463.50,1242.75,43.737657,-79.349718


In [14]:
dwelling_types = to_data.drop(['average_house_value', 'shelter_costs_owned', 'shelter_costs_rented'], axis=1)

In [46]:
neighborhoods = list(to_average_data['neighbourhood'].unique())

### Panel Visualization Functions

In [42]:
# Define Panel visualization functions
def neighbourhood_map():
    """Neighbourhood Map"""
   
    map_plot = px.scatter_mapbox(
    df_neighbourhood_locations,
    lat="lat",
    lon="lon",
    color="neighbourhood",
    zoom=10,
    width=1000,
    height=1000)

    return map_plot

def create_bar_chart(data, year, title, xlabel, ylabel, color):
    """
    Create a barplot based in the data argument.
    """
    bar_plot = data.loc[year].hvplot.bar(xlabel=xlabel, ylabel=ylabel, title=title, rot=90, color=color, height=500)
    
    return bar_plot

def create_line_chart(data, factor, title, xlabel, ylabel, color):
    """
    Create a line chart based in the data argument.
    """
    line_plot = data[factor].hvplot.line(xlabel=xlabel, ylabel=ylabel, title=title, color=color, height=500)
    
    return line_plot

def average_house_value():
    """Average house values per year."""
    
    # YOUR CODE HERE!

def average_value_by_neighbourhood():
    """Average house values by neighbourhood."""
    
    map_plot = px.scatter_mapbox(
    to_average_data_merged_location,
    lat="lat",
    lon="lon",
    size="average_house_value",
    color="neighbourhood",
    zoom=10,
    width=1000,
    height=1000)

    return map_plot

def create_interactive_bar_chart(area):
    chart_data = dwelling_types[dwelling_types['neighbourhood']==area].hvplot.bar(title=area, rot=90, height=500, width=1200)
    
    return chart_data

def average_house_value_snapshot():
    """Average house value for all Toronto's neighbourhoods per year."""

    chart =px.bar(to_data, x='neighbourhood', y='average_house_value', height=600, color='average_house_value')
    
    return chart

def top_most_expensive_neighbourhoods():
    """Top 10 most expensive neighbourhoods."""
    
    chart = top_10_neighborhoods['average_house_value'].hvplot.bar(title='Top 10 Neighborhoods By Cost', rot=90, height=500, width=1200)
    
    return chart

def sunburts_cost_analysis():
    """Sunburst chart to conduct a costs analysis of most expensive neighbourhoods in Toronto per year."""
    
    fig = px.sunburst(top_10_expensive_by_year, path=['year', 'neighbourhood'], values='average_house_value', height=1000)
    
    return fig

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [48]:
# Create a Title for the Dashboard
column= pn.Column("# Welcome to the Toronto Neighbourhood Analysis")
tabs = pn.Tabs(("neighbourhoods", neighbourhood_map),
              ("bar charts", pn.Column(pn.Row(create_bar_chart(yearly_dwelling_count_averaged_cost_removed, 2001, title='Dwelling Types in Toronto in 2001', xlabel='2001', ylabel='Dwelling Type Units', color='blue'),
                                             create_bar_chart(yearly_dwelling_count_averaged_cost_removed, 2006, title='Dwelling Types in Toronto in 2006', xlabel='2006', ylabel='Dwelling Type Units', color='red')),
                                      pn.Row(create_bar_chart(yearly_dwelling_count_averaged_cost_removed, 2011, title='Dwelling Types in Toronto in 2011', xlabel='2011', ylabel='Dwelling Type Units', color='orange'),
                                             create_bar_chart(yearly_dwelling_count_averaged_cost_removed, 2016, title='Dwelling Types in Toronto in 2016', xlabel='2016', ylabel='Dwelling Type Units', color='green')))),
              ("Average Values", average_value_by_neighbourhood()),
              ('interactive bar charts', interact(create_interactive_bar_chart, area=neighborhoods)))
tabs


Tabs
    [0] Column
        [0] Column()
        [1] Row
            [0] Plotly(Figure, name='interactive08681')
    [1] Column
        [0] Row
            [0] HoloViews(Bars)
            [1] HoloViews(Bars)
        [1] Row
            [0] HoloViews(Bars)
            [1] HoloViews(Bars)
    [2] Plotly(Figure, name='Average Values', visible=False)
    [3] Column
        [0] Column
            [0] Select(name='area', options=['Agincourt North', ...], value='Agincourt North')
        [1] Row
            [0] HoloViews(Bars, name='interactive08603')

## Serve the Panel Dashboard

In [ ]:
dashboard.servable()

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# neighbourhood_map().show()

In [ ]:
# create_bar_chart(data, title, xlabel, ylabel, color)

# # Bar chart for 2001
# create_bar_chart(df_dwelling_units.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Type Units", "red")

# # Bar chart for 2006
# create_bar_chart(df_dwelling_units.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "blue")

# # Bar chart for 2011
# create_bar_chart(df_dwelling_units.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "orange")

# # Bar chart for 2016
# create_bar_chart(df_dwelling_units.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "magenta")

In [ ]:
# create_line_chart(data, title, xlabel, ylabel, color)

# # Line chart for owned dwellings
# create_line_chart(df_avg_costs["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue")

# # Line chart for rented dwellings
# create_line_chart(df_avg_costs["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange")

In [ ]:
# average_house_value()

In [ ]:
# average_value_by_neighbourhood()

In [ ]:
# number_dwelling_types()

In [ ]:
# average_house_value_snapshot()

In [ ]:
# top_most_expensive_neighbourhoods()

In [ ]:
# sunburts_cost_analysis()